# Simple Tracking Demo

This notebook demonstrates basic object tracking using the soccer tracking pipeline.

## 1. Import Libraries

In [ ]:
import sys
from pathlib import Path
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Video, display

# Add src to path
sys.path.append('..')

from src.tracking import YOLOTracker, BotSortTracker
from src.data import VideoGenerator
from src.utils.visualization import draw_tracks
from src.utils.file_utils import download_file, ensure_dir

## 2. Download Sample Video

In [ ]:
# Ensure we have a sample video
sample_video_path = "../data/sample_videos/people.mp4"
sample_video_url = "https://github.com/mikel-brostrom/yolov8_tracking/raw/main/data/videos/people.mp4"

ensure_dir("../data/sample_videos")

if not Path(sample_video_path).exists():
    print("Downloading sample video...")
    success = download_file(sample_video_url, sample_video_path)
    if success:
        print("Sample video downloaded successfully!")
    else:
        print("Failed to download sample video.")
else:
    print("Sample video already exists.")

# Check video properties
if Path(sample_video_path).exists():
    cap = cv2.VideoCapture(sample_video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duration = frame_count / fps
    
    print(f"\nVideo properties:")
    print(f"Resolution: {width}x{height}")
    print(f"FPS: {fps}")
    print(f"Frames: {frame_count}")
    print(f"Duration: {duration:.2f} seconds")
    
    cap.release()

## 3. YOLO Tracker Demo

In [ ]:
# Initialize YOLO tracker
print("Initializing YOLO tracker...")
yolo_tracker = YOLOTracker(
    model_name='yolov8n.pt',
    confidence=0.3,
    device='auto'
)

print("YOLO tracker initialized successfully!")

In [ ]:
# Run YOLO tracking on sample video
output_video_yolo = "../output/sample_tracking_yolo.mp4"
ensure_dir("../output")

print("Running YOLO tracking...")
yolo_results = yolo_tracker.track_video(
    video_path=sample_video_path,
    output_path=output_video_yolo
)

print(f"YOLO tracking completed! Output saved to: {output_video_yolo}")
print(f"Processed {len(yolo_results)} frames")

# Display statistics
stats = yolo_tracker.get_statistics()
print(f"\nTracking Statistics:")
for key, value in stats.items():
    print(f"  {key}: {value:.2f}" if isinstance(value, float) else f"  {key}: {value}")

## 4. Process Individual Frames

In [ ]:
# Process and visualize individual frames
cap = cv2.VideoCapture(sample_video_path)
yolo_tracker.reset()  # Reset for fresh tracking

# Process first few frames and visualize
frames_to_show = [10, 30, 50, 70, 90]  # Frame numbers to visualize
visualized_frames = []

frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Run tracking
    tracks = yolo_tracker.update(None, frame)
    
    # Save specific frames for visualization
    if frame_idx in frames_to_show:
        annotated_frame = yolo_tracker.draw_tracks(frame.copy(), tracks)
        visualized_frames.append((frame_idx, annotated_frame, len(tracks)))
    
    frame_idx += 1
    
    # Stop early for demo
    if frame_idx > max(frames_to_show):
        break

cap.release()

# Display frames
fig, axes = plt.subplots(1, len(visualized_frames), figsize=(20, 4))
if len(visualized_frames) == 1:
    axes = [axes]

for i, (frame_num, frame, track_count) in enumerate(visualized_frames):
    # Convert BGR to RGB for matplotlib
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    axes[i].imshow(frame_rgb)
    axes[i].set_title(f'Frame {frame_num}\n{track_count} tracks')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## 5. Compare Detection vs Tracking

In [ ]:
# Compare detection-only vs tracking
cap = cv2.VideoCapture(sample_video_path)
yolo_tracker.reset()

# Process one frame to compare
cap.set(cv2.CAP_PROP_POS_FRAMES, 50)  # Go to frame 50
ret, frame = cap.read()

if ret:
    # Detection only
    detections = yolo_tracker.detect_only(frame)
    frame_detections = frame.copy()
    
    # Draw detections
    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        cv2.rectangle(frame_detections, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame_detections, f'{conf:.2f}', (x1, y1-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Tracking
    tracks = yolo_tracker.update(None, frame)
    frame_tracking = yolo_tracker.draw_tracks(frame.copy(), tracks)
    
    # Display comparison
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    ax1.imshow(cv2.cvtColor(frame_detections, cv2.COLOR_BGR2RGB))
    ax1.set_title(f'Detection Only\n{len(detections)} detections')
    ax1.axis('off')
    
    ax2.imshow(cv2.cvtColor(frame_tracking, cv2.COLOR_BGR2RGB))
    ax2.set_title(f'Tracking\n{len(tracks)} tracks')
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()

cap.release()

## 6. Display Output Video

In [ ]:
# Display the output video in the notebook
if Path(output_video_yolo).exists():
    print("Original video:")
    display(Video(sample_video_path, width=400))
    
    print("\nTracked video:")
    display(Video(output_video_yolo, width=400))
else:
    print("Output video not found. Please run the tracking cell above.")

## 7. Track Custom Video (Optional)

In [ ]:
# Upload and track your own video
try:
    from google.colab import files
    print("Upload a video file to track:")
    uploaded = files.upload()
    
    if uploaded:
        # Get the uploaded file
        uploaded_filename = list(uploaded.keys())[0]
        print(f"Processing uploaded video: {uploaded_filename}")
        
        # Track the uploaded video
        custom_output = f"../output/tracked_{uploaded_filename}"
        
        yolo_tracker.reset()
        custom_results = yolo_tracker.track_video(
            video_path=uploaded_filename,
            output_path=custom_output
        )
        
        print(f"Tracking completed! Output saved to: {custom_output}")
        
        # Display the result
        if Path(custom_output).exists():
            display(Video(custom_output, width=600))
        
except ImportError:
    print("File upload only available in Google Colab.")
    print("To track a custom video, place it in the data/sample_videos/ directory and modify the path above.")

## 8. Performance Analysis

In [ ]:
# Analyze tracking performance
import time

def benchmark_tracking(video_path, num_frames=100):
    """Benchmark tracking performance."""
    cap = cv2.VideoCapture(video_path)
    yolo_tracker.reset()
    
    start_time = time.time()
    frame_count = 0
    total_tracks = 0
    
    while frame_count < num_frames:
        ret, frame = cap.read()
        if not ret:
            break
        
        tracks = yolo_tracker.update(None, frame)
        total_tracks += len(tracks)
        frame_count += 1
    
    end_time = time.time()
    cap.release()
    
    processing_time = end_time - start_time
    fps = frame_count / processing_time
    avg_tracks = total_tracks / frame_count if frame_count > 0 else 0
    
    return {
        'frames_processed': frame_count,
        'processing_time': processing_time,
        'fps': fps,
        'avg_tracks_per_frame': avg_tracks,
        'total_tracks': total_tracks
    }

# Run benchmark
print("Running performance benchmark...")
benchmark_results = benchmark_tracking(sample_video_path, num_frames=100)

print("\n=== Performance Results ===")
for key, value in benchmark_results.items():
    if isinstance(value, float):
        print(f"{key}: {value:.2f}")
    else:
        print(f"{key}: {value}")

# Check if real-time performance is achieved
original_fps = 30  # Assuming 30 FPS original video
if benchmark_results['fps'] >= original_fps:
    print(f"\n✓ Real-time performance achieved! ({benchmark_results['fps']:.1f} FPS > {original_fps} FPS)")
else:
    print(f"\n⚠ Processing slower than real-time ({benchmark_results['fps']:.1f} FPS < {original_fps} FPS)")

## Summary

This demo showed:

1. **Basic YOLO tracking** on a sample video
2. **Frame-by-frame processing** and visualization
3. **Detection vs tracking comparison**
4. **Performance benchmarking**
5. **Custom video processing** capability

### Key Features Demonstrated:
- ✅ Automatic device detection (CPU/GPU)
- ✅ Real-time tracking performance
- ✅ Track ID consistency across frames
- ✅ Confidence scoring
- ✅ Video output generation

### Next Steps:
- Try `03_soccer_tracking_pipeline.ipynb` for advanced MOT dataset processing
- Experiment with different YOLO models (yolov8s, yolov8m, etc.)
- Test BotSort tracker for comparison
- Evaluate results using `04_evaluation_analysis.ipynb`